In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Part 1

Create a table of country names and worldbank country urls

In [36]:
url = "https://data.worldbank.org/country/"
response = requests.get(url)
response.status_code

200

In [37]:
def make_soup(url):
    response = requests.get(url)
    code = response.status_code
    assert ((code >= 200) & (code < 300))
    page = response.text
    soup = BeautifulSoup(page, 'lxml')
    return soup

In [38]:
soup = make_soup(url)

In [39]:
sections = soup.find_all('section')
atags = [x.find_all('a') for x in sections]
atag_list =  []
for result_set in atags:
    result_list = list(result_set)
    for result in result_list:
        atag_list.append(result)

In [40]:
country_dict = {'Country': [], 'URL': []}

for item in atag_list:
    country = item.text
    url = "https://data.worldbank.org" + item['href']
    country_dict['Country'].append(country)
    country_dict['URL'].append(url)

In [41]:
df_urls = pd.DataFrame(country_dict)

In [44]:
df_urls.head()

,Country,URL
0,Afghanistan,https://data.worldbank.org/country/afghanistan...
1,Albania,https://data.worldbank.org/country/albania?vie...
2,Algeria,https://data.worldbank.org/country/algeria?vie...
3,American Samoa,https://data.worldbank.org/country/american-sa...
4,Andorra,https://data.worldbank.org/country/andorra?vie...


In [77]:
df_urls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 2 columns):
Country    217 non-null object
URL        217 non-null object
dtypes: object(2)
memory usage: 3.5+ KB


In [62]:
df_urls.to_csv('country_urls.csv')

### Part 2

Create a function that uses a selenium webdriver to download country economic data using urls from part 1

In [43]:
from selenium import webdriver
import chromedriver_binary

In [46]:
driver = webdriver.Chrome()
driver.get(df_urls['URL'][0])

In [61]:
download_div = driver.find_element_by_class_name("download")
csv_download = download_div.find_element_by_xpath('.//a')
csv_download.click()
#download_div.get_attribute('innerHTML')
#download_div.find_element_by_xpath("//a").text

In [135]:
import time
import random

In [141]:
def scrape_country_data(driver, url):
    '''
    Downloads country data from the worldbank website
    
    :Args:
        - driver: a selenium webdriver object
        - url: URL for the country of interest i.e.
            'https://data.worldbank.org/country/afghanistan?view=chart'

    :Returns:
        - True if download successful. Downloaded files will appear in computer 'Download' file directory.
        - False if not successful.
    '''
    
    try:
        driver.get(url)
        download_div = driver.find_element_by_class_name("download")
        csv_download = download_div.find_element_by_xpath('.//a')
        csv_download.click()
        time.sleep(random.randint(1,3))
        return True
    except:
        time.sleep(random.randint(1,3))
        return False

### Part 3

Merge url table from part 1 with big mac index data

In [66]:
bm_index = pd.read_csv('data/big-mac-full-index.csv')

bm_index.tail()

In [88]:
# Rename the 'name' column to 'Country' to match df_urls

bm_index.rename(columns = {'name': 'Country'}, inplace = True)

In [89]:
bm_index['Country'].nunique()

# There are only 57 countries tracked by the big mac index

57

In [72]:
bm_index['date'] = pd.to_datetime(bm_index['date'])

In [90]:
bm_index[['date', 'Country']].groupby(by = 'date').agg('count')

# See how many countries are represented in each survey

,Country
date,
2000-04-01,28
2001-04-01,28
2002-04-01,32
2003-04-01,31
2004-05-01,39
2005-06-01,40
2006-01-01,31
2006-05-01,40
2007-01-01,40


In [91]:
# List of all the countries included in most recent survey of BM index

bm_index[bm_index['date'] == bm_index['date'][1273]]['Country']

1218    United Arab Emirates
1219               Argentina
1220               Australia
1221              Azerbaijan
1222                 Bahrain
1223                  Brazil
1224                  Canada
1225             Switzerland
1226                   Chile
1227                   China
1228                Colombia
1229              Costa Rica
1230          Czech Republic
1231                 Denmark
1232                   Egypt
1233               Euro area
1234                 Britain
1235               Guatemala
1236               Hong Kong
1237                Honduras
1238                 Croatia
1239                 Hungary
1240               Indonesia
1241                   India
1242                  Israel
1243                  Jordan
1244                   Japan
1245             South Korea
1246                  Kuwait
1247                 Lebanon
1248               Sri Lanka
1249                 Moldova
1250                  Mexico
1251                Malaysia
1252          

In [97]:
# Countries that are not included in the BM index

set(df_urls['Country']) - set(bm_index['Country'])

{'Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Armenia',
 'Aruba',
 'Austria',
 'Bahamas, The',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Channel Islands',
 'Comoros',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 "Cote d'Ivoire",
 'Cuba',
 'Curacao',
 'Cyprus',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt, Arab Rep.',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'French Polynesia',
 'Gabon',
 'Gambia, The',
 'Georgia',
 'Germany',
 'Ghana',
 'Gibraltar',
 'Greece',
 'Greenland',
 'Grenada',
 'Guam',
 'Guinea',
 'Guinea-Bissau

In [103]:
# Merge worldbank urls with BM Index

bmi_urls = bm_index.merge(df_urls, how = "left", on = "Country")

In [111]:
# need to do some manual tweaking to get urls for these countries merged

bmi_urls[(bmi_urls['URL'].isna()) & (bmi_urls['date'] == bmi_urls['date'][1273])]

,date,iso_a3,currency_code,Country,local_price,dollar_ex,dollar_price,USD_raw,EUR_raw,GBP_raw,JPY_raw,CNY_raw,GDP_dollar,adj_price,USD_adjusted,EUR_adjusted,GBP_adjusted,JPY_adjusted,CNY_adjusted,URL
1232,2019-07-09,EGY,EGP,Egypt,42.00,16.615000,2.527836,-0.560,-0.447,-0.384,-0.295,-0.171,2495.02000,2.769511,-0.206,-0.178,-0.061,0.058,-0.100,NaN
1233,2019-07-09,EUZ,EUR,Euro area,4.08,0.892339,4.572252,-0.203,0.000,0.115,0.275,0.499,37379.58307,4.119812,-0.035,0.000,0.142,0.287,0.095,NaN
1234,2019-07-09,GBR,GBP,Britain,3.29,0.802182,4.101314,-0.285,-0.103,0.000,0.144,0.345,39975.38000,4.220289,-0.155,-0.124,0.000,0.127,-0.041,NaN
1236,2019-07-09,HKG,HKD,Hong Kong,20.50,7.810500,2.624672,-0.543,-0.426,-0.360,-0.268,-0.140,46091.22000,4.457020,-0.488,-0.469,-0.394,-0.317,-0.419,NaN
1245,2019-07-09,KOR,KRW,South Korea,4500.00,1180.550000,3.811783,-0.336,-0.166,-0.071,0.063,0.250,29749.80000,3.824481,-0.133,-0.102,0.026,0.156,-0.017,NaN
1262,2019-07-09,RUS,RUB,Russia,130.00,63.838750,2.036381,-0.645,-0.555,-0.503,-0.432,-0.332,10961.99000,3.097248,-0.428,-0.408,-0.323,-0.238,-0.351,NaN
1268,2019-07-09,TWN,TWD,Taiwan,72.00,31.172000,2.309765,-0.598,-0.495,-0.437,-0.356,-0.243,24389.68000,3.617003,-0.445,-0.425,-0.343,-0.260,-0.370,NaN


In [126]:
df_urls.loc[df_urls['Country'] == 'Egypt, Arab Rep.','Country']

58    Egypt, Arab Rep.
Name: Country, dtype: object

In [127]:
# This will match up naming of countries appropriately
# The worldbank does not have data for Taiwan
# Also worth considering breaking out Euro area countries, will require manually finding BM prices

df_urls.loc[df_urls['Country'] == 'Egypt, Arab Rep.', 'Country'] = "Egypt"
df_urls.loc[df_urls['Country'] == 'United Kingdom', 'Country'] = "Britain"
df_urls.loc[df_urls['Country'] == 'Hong Kong SAR, China', 'Country'] = "Hong Kong"
df_urls.loc[df_urls['Country'] == 'Korea, Rep.', 'Country'] = "South Korea"
df_urls.loc[df_urls['Country'] == 'Russian Federation', 'Country'] = "Russia"

In [129]:
# Merge worldbank urls with BM Index again to bring in countries from above

bmi_urls = bm_index.merge(df_urls, how = "left", on = "Country")

In [130]:
# Now we have url for every country except 'Euro area' and 'Taiwan'

bmi_urls[(bmi_urls['URL'].isna()) & (bmi_urls['date'] == bmi_urls['date'][1273])]

,date,iso_a3,currency_code,Country,local_price,dollar_ex,dollar_price,USD_raw,EUR_raw,GBP_raw,JPY_raw,CNY_raw,GDP_dollar,adj_price,USD_adjusted,EUR_adjusted,GBP_adjusted,JPY_adjusted,CNY_adjusted,URL
1233,2019-07-09,EUZ,EUR,Euro area,4.08,0.892339,4.572252,-0.203,0.000,0.115,0.275,0.499,37379.58307,4.119812,-0.035,0.000,0.142,0.287,0.095,NaN
1268,2019-07-09,TWN,TWD,Taiwan,72.00,31.172000,2.309765,-0.598,-0.495,-0.437,-0.356,-0.243,24389.68000,3.617003,-0.445,-0.425,-0.343,-0.260,-0.370,NaN


### Part 4

download data from worldbank for countries in the BM index

In [143]:
url_list = list(bmi_urls[bmi_urls['date'] == bmi_urls['date'][1273]].dropna()['URL'])

In [144]:
country_list = list(bmi_urls[bmi_urls['date'] == bmi_urls['date'][1273]].dropna()['Country'])

In [145]:
driver = webdriver.Chrome()

for country, url in zip(country_list, url_list):
    success = scrape_country_data(driver, url)
    if success:
        print("Successfully downloaded data for " + country)
    else:
        print("Failed to download data for " + country)

Successfully downloaded data for Argentina
Successfully downloaded data for Australia
Successfully downloaded data for Brazil
Successfully downloaded data for Canada
Successfully downloaded data for Switzerland
Failed to download data for Chile
Successfully downloaded data for China
Successfully downloaded data for Colombia
Successfully downloaded data for Czech Republic
Successfully downloaded data for Denmark
Successfully downloaded data for Egypt
Successfully downloaded data for Britain
Successfully downloaded data for Hong Kong
Successfully downloaded data for Hungary
Successfully downloaded data for Indonesia
Successfully downloaded data for India
Successfully downloaded data for Israel
Successfully downloaded data for Japan
Successfully downloaded data for South Korea
Successfully downloaded data for Mexico
Successfully downloaded data for Malaysia
Successfully downloaded data for Norway
Successfully downloaded data for New Zealand
Successfully downloaded data for Pakistan
Succes

successfully downloaded data for all countries except Chile, will attempt to get that manually

In [150]:
len(bmi_urls[bmi_urls['date'] == bmi_urls['date'][1273]]['URL']) - len(url_list)

# still missing 19 countries

21

In [153]:
bmi_urls[bmi_urls['date'] == bmi_urls['date'][1273]].shape

(56, 20)

In [165]:
# when creating url list and country list I dropped too many rows
# dropna method was not in the proper place

set(bmi_urls[bmi_urls['date'] == bmi_urls['date'][1273]]['URL'].dropna()) - set(url_list)

{'https://data.worldbank.org/country/azerbaijan?view=chart',
 'https://data.worldbank.org/country/bahrain?view=chart',
 'https://data.worldbank.org/country/costa-rica?view=chart',
 'https://data.worldbank.org/country/croatia?view=chart',
 'https://data.worldbank.org/country/guatemala?view=chart',
 'https://data.worldbank.org/country/honduras?view=chart',
 'https://data.worldbank.org/country/jordan?view=chart',
 'https://data.worldbank.org/country/kuwait?view=chart',
 'https://data.worldbank.org/country/lebanon?view=chart',
 'https://data.worldbank.org/country/moldova?view=chart',
 'https://data.worldbank.org/country/nicaragua?view=chart',
 'https://data.worldbank.org/country/oman?view=chart',
 'https://data.worldbank.org/country/qatar?view=chart',
 'https://data.worldbank.org/country/romania?view=chart',
 'https://data.worldbank.org/country/sri-lanka?view=chart',
 'https://data.worldbank.org/country/ukraine?view=chart',
 'https://data.worldbank.org/country/united-arab-emirates?view=cha

In [163]:
# capture the remaining 19 countries in the lists below

country_list2 = list(bmi_urls[['date', 'Country', 'URL']][bmi_urls['date'] == bmi_urls['date'][1273]].dropna()['Country'])
url_list2 = list(bmi_urls[['date', 'Country', 'URL']][bmi_urls['date'] == bmi_urls['date'][1273]].dropna()['URL'])
excluded_countries = set(country_list2) - set(country_list)
excluded_urls = set(url_list2) - set(url_list)
print(len(excluded_countries), len(excluded_urls))

19 19


In [167]:
# scrape data for remaining countries
driver = webdriver.Chrome()

for country, url in zip(excluded_countries, excluded_urls):
    success = scrape_country_data(driver, url)
    if success:
        print("Successfully downloaded data for " + country)
    else:
        print("Failed to download data for " + country)

Successfully downloaded data for Kuwait
Successfully downloaded data for Sri Lanka
Successfully downloaded data for Uruguay
Successfully downloaded data for Ukraine
Successfully downloaded data for Honduras
Successfully downloaded data for Vietnam
Successfully downloaded data for Moldova
Successfully downloaded data for Azerbaijan
Successfully downloaded data for Romania
Successfully downloaded data for Bahrain
Successfully downloaded data for Costa Rica
Successfully downloaded data for Lebanon
Successfully downloaded data for Croatia
Successfully downloaded data for Nicaragua
Successfully downloaded data for Jordan
Successfully downloaded data for Oman
Successfully downloaded data for Qatar
Successfully downloaded data for United Arab Emirates
Successfully downloaded data for Guatemala


### Part 5

Now that I have a model for the BMI. Lets scrape data for the rest of the countries in order to predict which ones would hypothetically have the most / least expensive big macs.

In [178]:
# Create a list of the remaining country URLs
remaining_urls = list(set(df_urls['URL']) - set(bmi_urls[bmi_urls['date'] == bmi_urls['date'][1273]]['URL']))

In [180]:
# Scrape the data from the worldbank

driver = webdriver.Chrome()

for i, url in enumerate(remaining_urls):
    success = scrape_country_data(driver, url)
    if success:
        print("Successfully downloaded data for index " + str(i))
    else:
        print("Failed to download data for index " + str(i))

Successfully downloaded data for index 0
Successfully downloaded data for index 1
Successfully downloaded data for index 2
Successfully downloaded data for index 3
Successfully downloaded data for index 4
Successfully downloaded data for index 5
Successfully downloaded data for index 6
Successfully downloaded data for index 7
Successfully downloaded data for index 8
Successfully downloaded data for index 9
Successfully downloaded data for index 10
Successfully downloaded data for index 11
Successfully downloaded data for index 12
Successfully downloaded data for index 13
Successfully downloaded data for index 14
Successfully downloaded data for index 15
Successfully downloaded data for index 16
Successfully downloaded data for index 17
Successfully downloaded data for index 18
Successfully downloaded data for index 19
Successfully downloaded data for index 20
Successfully downloaded data for index 21
Successfully downloaded data for index 22
Successfully downloaded data for index 23
Su

In [181]:
# Check out the url that failed and download manually
remaining_urls[117]

'https://data.worldbank.org/country/cyprus?view=chart'